<a href="https://colab.research.google.com/github/lala991204/ML-self-study/blob/master/8_5_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 지도학습 기반 감성 분석 실습 - IMDB 영화평

In [2]:
# 빈 파일 여부 파악(False=>빈 파일 x)
import os
os.stat('/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/labeledTrainData.tsv').st_size==0

False

In [3]:
import pandas as pd

review_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/github/ML-self-study/labeledTrainData.tsv', header=0, sep="\t", quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [ ]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [4]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review']= review_df['review'].str.replace('<br />', ' ')

# 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x: re.sub("[^a-zA-Z]", " ", x))

In [5]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id', 'sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, 
                                                    random_state=156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization 수행
# LogisticRegression의 C는 10으로 설정.
pipeline = Pipeline([
                     ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
                     ('lr_clf', LogisticRegression(C=10))
                     ])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행
# predict_proba()는 roc_auc를 위해 수행함
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                 roc_auc_score(y_test, pred_probs)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


예측 정확도는 0.8860, ROC-AUC는 0.9503


In [7]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
                     ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                     ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8936, ROC-AUC는 0.9598


# 비지도학습 기반 감성 분석 소개

## SentiWordNet을 이용한 Sentiment Analysis

WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해

In [8]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [9]:
from nltk.corpus import wordnet as wn    
# corpus(말뭉치)는 자연어 분석 작업을 위해 만든 샘플 문서 집합
# wordnet은 영어의 의미 어휘목록이고, 워드넷은 영어 단어를 'synset'이라는 유의어 집단으로 분류하여 간략하고 일반적인 정의 제공하고,
# 이러한 어휘목록 사이의 다양한 의미 관계 기록함.

term = 'present'

# 'present'라는 단어로 wordnet의 synsets 생성
synsets = wn.synsets(term)
print('synsets() 반환 type :', type(synsets))
print('synsets() 반환 값 갯수:', len(synsets))
print('synsets() 반환 값 :', synsets)

synsets() 반환 type : <class 'list'>
synsets() 반환 값 갯수: 18
synsets() 반환 값 : [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


In [10]:
for synset in synsets:
    print('##### Synset name: ', synset.name(), '#####')
    print('POS: ', synset.lexname())
    print('Definition: ', synset.definition())
    print('Lemmas: ', synset.lemma_names())

##### Synset name:  present.n.01 #####
POS:  noun.time
Definition:  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemmas:  ['present', 'nowadays']
##### Synset name:  present.n.02 #####
POS:  noun.possession
Definition:  something presented as a gift
Lemmas:  ['present']
##### Synset name:  present.n.03 #####
POS:  noun.communication
Definition:  a verb tense that expresses actions or states at the time of speaking
Lemmas:  ['present', 'present_tense']
##### Synset name:  show.v.01 #####
POS:  verb.perception
Definition:  give an exhibition of to an interested audience
Lemmas:  ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name:  present.v.02 #####
POS:  verb.communication
Definition:  bring forward and present to the mind
Lemmas:  ['present', 'represent', 'lay_out']
##### Synset name:  stage.v.01 #####
POS:  verb.creation
Definition:  perform (a play), especially on a stage
Lemmas:  ['stage', 'present', 're

In [14]:
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]
entity_names = [entity.name().split('.')[0] for entity in entities]
entity_names

['tree', 'lion', 'tiger', 'cat', 'dog']

In [24]:
tree

Synset('tree.n.01')

In [17]:
tree.name().split('.')

['tree', 'n', '01']

In [18]:
# 단어별 synset들을 iteration하면서 다른 단어들의 synset과 유사도 측정
similarities = []
for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity), 2) for compared_entity in entities]
    similarities.append(similarity)

# 개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame형태로 저장
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [20]:
tree.path_similarity(lion)

0.07142857142857142

In [21]:
similarities

[[1.0, 0.07, 0.07, 0.08, 0.12],
 [0.07, 1.0, 0.33, 0.25, 0.17],
 [0.07, 0.33, 1.0, 0.25, 0.17],
 [0.08, 0.25, 0.25, 1.0, 0.2],
 [0.12, 0.17, 0.17, 0.2, 1.0]]

SentiWordNet은 반지도학습 기법을 바탕으로 WordNet의 Synset에 감성스코어를 매긴 어휘사전이다.

In [22]:
import nltk
from nltk.corpus import sentiwordnet as swn

senti_synsets = list(swn.senti_synsets('slow'))
print('senti_synsets() 반환 type: ', type(senti_synsets))
print('senti_synsets() 반환 값 갯수: ', len(senti_synsets))
print('senti_synsets() 반환 값: ', senti_synsets)

senti_synsets() 반환 type:  <class 'list'>
senti_synsets() 반환 값 갯수:  11
senti_synsets() 반환 값:  [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


감성스코어는 긍정(P), 부정(N), 중립(O)으로 구성되며, 한 synset에 대해 P + N + O = 1 이 됨. 만약 P = 1이면 그 synset은 강렬하게 긍정적인 의미인것이고, 반대로 N = 1이면 강렬하게 부정적인 의미임. P =0, N = 0이라면 O=1이 되며, 그 synset은 완전 중립적인 것임.

In [23]:
import nltk
from nltk.corpus import sentiwordnet as wn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())
print('\n')
fabulous = swn.senti_synset('fabulous.a.01')     # 뜻: 굉장한
print('fabulous 긍정감성 지수: ',fabulous .pos_score())
print('fabulous 부정감성 지수: ',fabulous .neg_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125


In [26]:
print('fabulous 객관성 지수: ', fabulous.obj_score())

fabulous 객관성 지수:  0.0


In [27]:
from nltk.corpus import wordnet as wn

# 간단한 NTLK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.VERB
    return

str.startswith(str or tuple)형식으로 사용하면 되고, 반환 값으로는 True, False를 반환함. 인자에 해당하는 부분이 str에 적은 첫 부분에 해당한다면 True를 반환함.

In [31]:
penn_to_wn('NN')   

'n'

In [40]:
penn_to_wn('JR')

'a'

In [41]:
penn_to_wn('RJ')

'v'

In [42]:
penn_to_wn('TJ')

In [35]:
wn.NOUN

'n'

In [36]:
wn.ADJ

'a'

In [37]:
wn.VERB

'v'

if not에서
not 0, not None, not ''(공백)은 모두 True가 됨.

In [44]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):

    # 감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0

    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)

    # 분해된 문장별 단어 토큰 -> 품사 태깅 후 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:

        # NTLK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:

            # WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):  
                continue         # 아래를 실행하지 말고 반복문으로 다시 돌아가시오.
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)     
            if not lemma:   
                continue
            
            # 어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:  
                continue
            
            # sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            # 모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count += 1

    if not tokens_count:  
        return 0
    
    # 총 score이 0이상일 경우 긍정(Positive) 1, 그렇지 않을 경우 부정(Negative) 0 반환
    if sentiment >= 0:
         return 1
    
    return 0

            

In [45]:
review_df['preds'] = review_df['review'].apply(lambda x : swn_polarity(x))
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [46]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score 
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix( y_target, preds))
print("정확도:", np.round(accuracy_score(y_target , preds), 4))
print("정밀도:", np.round(precision_score(y_target , preds),4))
print("재현율:", np.round(recall_score(y_target, preds), 4))

[[7301 5199]
 [3187 9313]]
정확도: 0.6646
정밀도: 0.6417
재현율: 0.745


## VADER lexicon을 이용한 Sentiment Analysis

'neg'는 부정 감성 지수, 'neu'(neutrality)는 중립적인 감성 지수, 'pos'는 긍정 감성 지수, 그리고 compound는 neg, neu, pos score를 적절히 조합해 -1에서 1사이의 감성 지수를 표현한 값이다.

In [47]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


compound score를 기반으로 부정 감성 또는 긍정 감성 여부를 결정하는데, 보통 0.1 이상이면 긍정 감성, 그 이하이면 부정감성으로 판단하나 상황에 따라 이 임계값을 적절히 조정해 예측 성능을 조절한다.

In [48]:
def vader_polarity(review,threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    # compound 값에 기반하여 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환 
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

# apply lambda 식을 이용하여 레코드별로 vader_polarity( )를 수행하고 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply( lambda x : vader_polarity(x, 0.1) )
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print(confusion_matrix( y_target, vader_preds))
print("정확도:", np.round(accuracy_score(y_target , vader_preds),4))
print("정밀도:", np.round(precision_score(y_target , vader_preds),4))
print("재현율:", np.round(recall_score(y_target, vader_preds),4))

[[ 6736  5764]
 [ 1867 10633]]
정확도: 0.6948
정밀도: 0.6485
재현율: 0.8506
